In [3]:
import csv
from rdflib import Graph, Literal, Namespace, RDF,RDFS, URIRef

In [2]:
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 4.4 MB/s eta 0:00:00


In [4]:
# Defining the ontology
ont = Namespace('http://www.semanticweb.org/tai/ontologies/2023/2/untitled-ontology-20#')
movie_class = ont.Movie
genre_class = ont.Genre
actor_class = ont.Actor
director_class = ont.Director
writer_class = ont.Writer

In [5]:
graph = Graph()

In [15]:
import io
with io.open('/content/movies.csv', newline='', encoding= 'latin-1') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # Create a new URI for the movie
        movie_uri = URIRef(ont + row['Movie'].replace(' ', '_'))

        # Add triples for the genres
        for genre in row['Genre'].split(','):
            genre_uri = URIRef(ont + genre.replace(' ', ''))
            graph.add((genre_uri, RDF.type, genre_class))
            graph.add((movie_uri, RDF.type, genre_uri))

        # Add triples for the actors
        for actor in row['Actors'].split(','):
            actor_uri = URIRef(ont + actor.replace(' ', ''))
            graph.add((movie_uri, ont.hasActor, actor_uri))
            graph.add((actor_uri, RDF.type, actor_class))

        # Add triples for the director
        for director in row['Director'].split(','):
            director_uri = URIRef(ont + row['Director'].replace(' ', ''))
            graph.add((movie_uri, ont.isDirectedby, director_uri))
            graph.add((director_uri, RDF.type, director_class))

        # Add triples for the writers
        for writer in row['Writer'].split(','):
            writer_uri = URIRef(ont + writer.replace(' ', ''))
            graph.add((movie_uri, ont.isWrittenby, writer_uri))
            graph.add((writer_uri, RDF.type, writer_class))

        # Add triples for the country, language, title, and year
        graph.add((movie_uri, ont.country, Literal(row['Country'])))
        graph.add((movie_uri, ont.language, Literal(row['Language'])))
        graph.add((movie_uri, ont.title, Literal(row['Movie'])))
        graph.add((movie_uri, ont.year, Literal(int(row['Year']))))

In [19]:
#printing the file saved in RDF

In [18]:
with open('movies.rdf', 'wb') as f:
    f.write(graph.serialize(format='xml').encode())